In [37]:
import pandas as pd
import json

In [161]:
base_path = '../../data/'

In [162]:
input_path = base_path + 'max-bitard-chat/result.json'

In [163]:
out_path = base_path + 'max-bitard-chat/max-messages.tsv'
out_path_seq = base_path + 'max-bitard-chat/max-sequences.tsv'

In [40]:
with open(input_path, 'r') as f:
    data = json.load(f)

In [41]:
len(data['messages'])

27796

In [57]:
df_raw = pd.DataFrame.from_records((x for x in data['messages'] if x['type']=='message'))

In [58]:
df_raw.fillna({'forwarded_from': '', 'via_bot': ''}, inplace=True)

In [59]:
target_user_id = 'user516433742'

In [83]:
df = (
    df_raw
    .query('forwarded_from=="" and via_bot ==""')
    .query('text!=""')
    .assign(
        dt=lambda x: pd.to_datetime(x['date']),
        is_max=lambda x: x['from_id'] == target_user_id
    )
    .sort_values(['dt'])
)

In [84]:
df.is_max.value_counts()

True     12229
False     9543
Name: is_max, dtype: int64

In [140]:
df.apply(len, axis=1)

0        34
3        34
4        34
5        34
6        34
         ..
27724    34
27725    34
27726    34
27727    34
27730    34
Length: 21772, dtype: int64

In [155]:
def aggregate_seq(messages):
    return ' xxmessage '.join(messages)

In [156]:
df['last_message_td'] = df.dt-df.dt.shift(1)
df['is_seq_start'] = (df['last_message_td']>pd.Timedelta(3, 'min'))
df['seq_id'] = df.is_seq_start.cumsum()
df['target_text'] = df.text.apply(simplify_message)
df['target_text'] = df.apply(lambda x: 'xxmax ' + x.target_text if x.is_max and x.target_text!='' else x.target_text, axis=1)

In [167]:
df_seq = df[df.target_text!=''].groupby('seq_id')[['target_text']].agg(aggregate_seq)

In [168]:
df_seq.reset_index().to_csv(out_path_seq, sep='\t', index=False)

In [148]:
target_columns = [
    'id', 'dt', 'from', 'from_id', 'text', 'target_text',
       'reply_to_message_id', 'performer','is_max','seq_id']

In [149]:
# _target_columns = [
#     'id', 'dt', 'from', 'from_id', 'text',
#        'media_type', 'sticker_emoji',
#        'reply_to_message_id', 'performer','is_max', 'last_message_td', 'is_seq_start', 'seq_id']

In [150]:
df.iloc[2000:2030, :].T

,3808,3809,3810,3811,3812,3813,3815,3814,3816,3817,...,3834,3835,3836,3837,3838,3839,3840,3841,3842,3843
id,-999880072,-999880071,-999880070,-999879869,-999879868,-999878368,-999876285,-999876286,-999876284,-999876283,...,-999875090,-999875065,-999874907,-999874143,-999874044,-999874043,-999874042,-999874040,-999874039,-999874038
type,message,message,message,message,message,message,message,message,message,message,...,message,message,message,message,message,message,message,message,message,message
date,2020-02-07T11:40:16,2020-02-07T11:40:21,2020-02-07T11:40:54,2020-02-10T04:51:55,2020-02-10T04:51:55,2020-02-20T12:57:01,2020-03-05T09:14:27,2020-03-05T09:14:27,2020-03-05T09:14:36,2020-03-05T09:14:43,...,2020-03-13T15:46:34,2020-03-13T16:41:19,2020-03-15T08:58:35,2020-03-17T09:34:40,2020-03-17T11:31:09,2020-03-17T11:31:26,2020-03-17T11:31:31,2020-03-17T11:31:45,2020-03-17T11:32:03,2020-03-17T11:35:48
date_unixtime,1581072016,1581072021,1581072054,1581306715,1581306715,1582199821,1583396067,1583396067,1583396076,1583396083,...,1584110794,1584114079,1584259115,1584434080,1584441069,1584441086,1584441091,1584441105,1584441123,1584441348
from,Kovriga Hentai,Mad Max,Mad Max,Mad Max,Sublime Bot,Mad Max,Sublime Bot,Mad Max,Mad Max,Mad Max,...,Mad Max,Spinifer,Mad Max,Mad Max,Spinifer,Spinifer,Mad Max,Spinifer,Mad Max,Mad Max
from_id,user941686508,user516433742,user516433742,user516433742,user232117096,user516433742,user232117096,user516433742,user516433742,user516433742,...,user516433742,user782553673,user516433742,user516433742,user782553673,user782553673,user516433742,user782553673,user516433742,user516433742
text,"Да я знаю, что такое афазия",Зазора озеро пешка звонка сервер дополз хаты в...,"Телефон генерирует рандомные слова в чат, режи...","[{'type': 'bot_command', 'text': '/pidor@Subli...","[Зарегистрированных в игру еще нет, а значит ,...","Задам лучше тут этот вопрос, кто-то слышал что...","[Зарегистрированных в игру еще нет, а значит ,...","[{'type': 'bot_command', 'text': '/pidor@Subli...",я опять пидор дня,"[{'type': 'bot_command', 'text': '/pidor@Subli...",...,Даёшь запрещенные мемасы,А ты лижешь?,Кто бы мне дал то,"Сколько мы будем терпеть, скажи? Им эти мирные...",Если тебя полюбят то дадут,А если полюбишь ты то отлижешь,Никто меня не полюбит,Не отчаивайся,да мне уже как-то безразлично,"я сам прекрасно знаю что я стремный парень, чё..."
file,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
thumbnail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
media_type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
some_text = df[df.id==-999727822].text.values[0]

In [49]:
some_str = df[df.id==-999764807].text.values[0]

In [69]:
type(some_str)

str

In [70]:
type(some_text)

list

In [77]:
some_text

[{'type': 'mention', 'text': '@Idzuev'},
 ' А ты в курсе что Ельцин это был "Путин начало?"']

In [141]:
df.sample(5).T

,22000,11595,19960,12661,15272
id,4461,-999746691,2415,-999743370,-999735216
type,message,message,message,message,message
date,2022-04-26T10:29:09,2021-12-09T11:48:13,2022-04-04T12:09:14,2021-12-21T06:05:42,2022-01-25T05:12:55
date_unixtime,1650961749,1639046893,1649066954,1640063142,1643083975
from,Mad Max,Mad Max,Mad Max,Mad Max,Mad Max
from_id,user516433742,user516433742,user516433742,user516433742,user516433742
text,кровавую цену,что он тот в кого постоянно тыкают что вот тот...,и довести до точки эвакуации,в особо крупном,Как вытащить человека из жопы?
file,NaN,NaN,NaN,NaN,NaN
thumbnail,NaN,NaN,NaN,NaN,NaN
media_type,NaN,NaN,NaN,NaN,NaN


In [100]:
" ".join([])

''

In [133]:
def simplify_message(message):
    if type(message) == list:
        res_list = []
        for s in message:
            if type(s) == str:
                res_list.append(s)
            elif type(s) == dict:
                if s['type'] in ('mention', 'mention_name'):
                    res_list.append('user_mention_tok')
                elif s['type'] in ('bot_command', 'link', 'email', 'phone', 'code'):
                    pass
                elif s['type'] in ('hashtag', 'underline', 'strikethrough', 'bold'):
                    res_list.append(s['text'])
                else:
                    print(message)
                    print(s)
                    raise Exception(f'There is not handler for type {s["type"]} in subitem')
            else:
                print(message)
                print(s)
                raise Exception(f'There is not handler for type {type(s)} in subitem')
        return ' '.join(res_list)
    if type(message) == str:
        return message
    raise Exception(f'There is not handler for type {type(message)}')

In [50]:
simplify_message(some_text)

'user_mention_tok  А ты в курсе что Ельцин это был "Путин начало?"'

In [52]:
df['target_text'] = df.text.apply(simplify_message)

In [53]:
df.columns

Index(['id', 'type', 'date', 'date_unixtime', 'from', 'from_id', 'text',
       'file', 'thumbnail', 'media_type', 'sticker_emoji', 'width', 'height',
       'photo', 'forwarded_from', 'mime_type', 'duration_seconds',
       'reply_to_message_id', 'performer', 'title', 'via_bot', 'edited',
       'edited_unixtime', 'game_title', 'game_description', 'game_link',
       'poll', 'target_text'],
      dtype='object')

In [54]:
df[['id', 'date', 'date_unixtime', 'target_text']].sample(10)

,id,date,date_unixtime,target_text
25536,8012,2022-05-27T11:36:13,1653644173,Я когда устраивался такого маразма-то не было
26339,8819,2022-06-06T13:14:57,1654514097,"если прыгать в кровать, можно вызвать девочку ..."
13836,-999737905,2022-01-17T12:07:07,1642417627,Высад
24864,7338,2022-05-24T10:13:23,1653380003,сек я тут сделю штуку
19140,1585,2022-03-24T04:50:26,1648093826,они обсераются потому что им данные пиздабольс...
27101,9581,2022-06-10T07:18:30,1654838310,вы там не видите но у него пека ещё за кадром ...
4404,-999785400,2021-04-19T06:19:32,1618805972,хорошо
14557,-999736655,2022-01-20T12:07:45,1642676865,он и у Путина есть)
21366,3824,2022-04-15T07:24:14,1650000254,персонажи взяты некоторые из первой игры и лок...
16311,-999732480,2022-02-02T05:25:13,1643775913,Сбер просто красавчик хули


In [140]:
df[['id', 'date', 'date_unixtime', 'target_text']].to_csv(out_path, sep='\t', index=False)

### Try to open pickle vocab

In [1]:
import pickle

In [2]:
with open('../max-flood-bot/models/max_bot_lm_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [3]:
len(vocab)

3808

In [5]:
from requests import get

In [11]:
url = 'http://localhost:8001'
resp = get(url)

In [12]:
resp.json()

{'data': 'Hello, World!'}

In [20]:
predict_url = url + '/predict'

In [33]:
resp = get(predict_url, json=dict(text='У меня встал', n_words=11))

In [34]:
resp

<Response [200]>

In [35]:
resp.json()

{'predicted_text': 'у меня встал вопрос , в конце прошлого года назад , в смысле ,'}